<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

In [5]:

import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium

import numpy as np
import pandas as pd
import geocoder
import requests 
from bs4 import BeautifulSoup 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

<h3>Scraping the Wikipedia page<h3>

In [6]:
website_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [7]:
r = requests.get(website_url) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 
print("Scraped and got the data") 

Scraped and got the data


<h3>Wragling and Cleaning the data</h3>

In [8]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
selected = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            selected = False
            if columnNum == 1:
                if selected == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    selected = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1

print('Data cleaned')

Data cleaned


<h3>Creating dataframe<h3>

In [9]:
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
lat_lng_coords = None

neighborhood_df = pd.DataFrame(columns=column_names)

for data in range(len(neighborhoods)):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng
    
    neighborhood_df = neighborhood_df.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)
    
neighborhood_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752420,-79.329242
1,M4A,North York,Victoria Village,43.730600,-79.313265
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
3,M6A,North York,Lawrence Heights,43.723270,-79.451286
4,M6A,North York,Lawrence Manor,43.723270,-79.451286
5,M7A,Queen's Park,Queen's Park,43.661150,-79.391715
6,M9A,Queen's Park,Queen's Park,43.662299,-79.528195
7,M1B,Scarborough,Rouge,43.811525,-79.195517
8,M1B,Scarborough,Malvern,43.811525,-79.195517
9,M3B,North York,Don Mills North,43.749055,-79.362227


In [10]:
neighborhood_df.shape

(210, 5)

<h3> Fetching From Foursquare API</h3>

In [13]:
VERSION = '20180605' # Foursquare API version

neighborhood_name = neighborhood_df.loc[0, 'Neighborhood'] # neighborhood name
neighborhood_latitude = neighborhood_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhood_df.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API
CLIENT_ID = '3X1QIGDCUX3QU31VA5MIWGB54V1RFKR3I2EF02JLVSNNYZVO' # your Foursquare ID
CLIENT_SECRET = 'NNJHGVZJ4FMUV5ARX0NN1HIEWDIO2DIQPPZI4XHNABXRLB1I' # your Foursquare Secret

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
     CLIENT_ID, 
     CLIENT_SECRET, 
     VERSION, 
     neighborhood_latitude, 
     neighborhood_longitude, 
     radius, 
     LIMIT)

results = requests.get(url).json()
results

Latitude and longitude values of Parkwoods are 43.75242000000003, -79.32924245299995.


{'meta': {'code': 200, 'requestId': '5dd659b3df2774001c755b17'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.75692000450003,
    'lng': -79.32302427998279},
   'sw': {'lat': 43.74791999550003, 'lng': -79.33546062601711}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs

In [14]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

**Cleaning JSON and structuring it into df**

In [16]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114
2,GTA Restoration,Fireworks Store,43.753396,-79.333477


In [17]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


<h3> Exploring Neighborhoods in Toronto</h3>

Function to process neighborhood in Toronto

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Creating DF

In [22]:
toronto_venues = getNearbyVenues(names=neighborhoods,
                                 latitudes=neighborhood_df['Latitude'],
                                 longitudes=neighborhood_df['Longitude'])

Parkwoods
Victoria Village
Harbourfront
Lawrence Heights
Lawrence Manor
Queen's Park
Queen's Park
Rouge
Malvern
Don Mills North

Woodbine Gardens
Parkview Hill
Ryerson

Garden District

Glencairn

Cloverdale

Islington
Martin Grove

Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South

Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens

Eringate

Markland Wood
Old Burnhamthorpe

Guildwood

Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks

Woburn
Leaside
Central Bay Street

Christie

Cedarbrae

Hillcrest Village
Bathurst Manor
Downsview North

Wilson Heights
Thorncliffe Park
Adelaide

King

Richmond

Dovercourt Village
Dufferin

Scarborough Village
Fairview

Henry Farm
Oriole

Northwood Park
York University
East Toronto
Harbourfront East

Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park

Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East

The Danforth West

Riverdal

In [23]:
print(toronto_venues.shape)
toronto_venues.head()

(4787, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75242,-79.329242,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.75242,-79.329242,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.75242,-79.329242,GTA Restoration,43.753396,-79.333477,Fireworks Store
3,Victoria Village,43.73060,-79.313265,Wigmore Park,43.731023,-79.310771,Park
4,Victoria Village,43.73060,-79.313265,Memories of Africa,43.726602,-79.312427,Grocery Store


Venues for each neighborhood

In [24]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Agincourt,15,15,15,15,15,15
Agincourt North,2,2,2,2,2,2
Albion Gardens,13,13,13,13,13,13
Alderwood,4,4,4,4,4,4
Bathurst Quay,69,69,69,69,69,69
Bayview Village,4,4,4,4,4,4
Beaumond Heights,13,13,13,13,13,13
Bedford Park,21,21,21,21,21,21


Unique Category

In [25]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 259 uniques categories.


<h3> Analysing the Neighborhood</h3>

In [26]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [32]:
print('Shape of df : ',toronto_onehot.shape)

Shape of df :  (4787, 259)


Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [33]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Adelaide,0.000000,0.000000,0.00,0.030000,0.000000,0.010000,0.000000,0.030000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.030000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.020000,0.000000,0.000000,0.010000,0.020000,0.000000,0.000000,0.000000

In [34]:
toronto_grouped.shape


(200, 259)

printing each neighborhood along with the top 5 most common venues

In [35]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide
----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.06
2                Hotel  0.05
3           Steakhouse  0.04
4  Japanese Restaurant  0.03


----Agincourt----
                venue  freq
0       Shopping Mall  0.13
1  Chinese Restaurant  0.13
2         Supermarket  0.13
3                Pool  0.07
4      Discount Store  0.07


----Agincourt North----
                      venue  freq
0                  Pharmacy   1.0
1               Yoga Studio   0.0
2   New American Restaurant   0.0
3  Mediterranean Restaurant   0.0
4               Men's Store   0.0


----Albion Gardens
----
                  venue  freq
0         Grocery Store  0.15
1           Pizza Place  0.08
2                  Park  0.08
3   Fried Chicken Joint  0.08
4  Fast Food Restaurant  0.08


----Alderwood----
                 venue  freq
0       Sandwich Place  0.25
1    Convenience Store  0.25
2                  Pub  0.25
3                  Gym  0.25
4  Monument / Landm

sorting the venues in descending order

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

creating the new dataframe and display the top 10 venues for each neighborhood.

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Hotel,Steakhouse,Burger Joint,Bakery,Gastropub,Asian Restaurant,Bar,Restaurant
1,Agincourt,Chinese Restaurant,Shopping Mall,Supermarket,Sushi Restaurant,Discount Store,Bakery,Badminton Court,Bubble Tea Shop,Hong Kong Restaurant,Vietnamese Restaurant
2,Agincourt North,Pharmacy,Women's Store,Donut Shop,Fish & Chips Shop,Fireworks Store,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
3,Albion Gardens,Grocery Store,Pizza Place,Park,Coffee Shop,Sandwich Place,Fast Food Restaurant,Liquor Store,Fried Chicken Joint,Beer Store,Pharmacy
4,Alderwood,Convenience Store,Sandwich Place,Gym,Pub,Farmers Market,Farm,Falafel Restaurant,Event Space,Fast Food Restaurant,Field


<h3>Clustering Neighborhoods</h3>

Runing k-means to cluster the neighborhood into 5 clusters.

In [38]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([4, 4, 1, 4, 4, 4, 0, 4, 4, 4], dtype=int32)

creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [ ]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [46]:

toronto_merged = neighborhood_df

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752420,-79.329242,0.0,Fireworks Store,Food & Drink Shop,Park,Women's Store,Donut Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
1,M4A,North York,Victoria Village,43.730600,-79.313265,3.0,Park,Grocery Store,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Women's Store,Dog Run
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166,4.0,Coffee Shop,Bakery,Theater,Boat or Ferry,Gym / Fitness Center,Italian Restaurant,Hotel,Gym Pool,Ice Cream Shop,Mexican Restaurant
3,M6A,North York,Lawrence Heights,43.723270,-79.451286,4.0,Clothing Store,Furniture / Home Store,Cosmetics Shop,Sporting Goods Shop,American Restaurant,Men's Store,Toy / Game Store,Food Court,Lingerie Store,Café
4,M6A,North York,Lawrence Manor,43.723270,-79.451286,4.0,Clothing Store,Furniture / Home Store,Cosmetics Shop,Sporting Goods Shop,American Restaurant,Men's Store,Toy / Game Store,Food Court,Lingerie Store,Café


visualize the resulting clusters


In [55]:
map_clusters = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    
    if np.isnan(cluster):
        cluster=0
    
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3>Examine Clusters</h3>

Cluster 1

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0.0,Fireworks Store,Food & Drink Shop,Park,Women's Store,Donut Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
32,Scarborough,0.0,Construction & Landscaping,Park,Gym / Fitness Center,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Dog Run,Fireworks Store
33,Scarborough,0.0,Construction & Landscaping,Park,Gym / Fitness Center,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Dog Run,Fireworks Store
34,Scarborough,0.0,Construction & Landscaping,Park,Gym / Fitness Center,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Dog Run,Fireworks Store
37,York,0.0,Park,Women's Store,Gym,Market,Fast Food Restaurant,Mexican Restaurant,Beer Store,Bakery,Sporting Goods Shop,Department Store
59,East York,0.0,Convenience Store,Park,Farmers Market,Italian Restaurant,Cuban Restaurant,Dumpling Restaurant,Fish & Chips Shop,Fireworks Store,Field,Fast Food Restaurant
60,Downtown Toronto,0.0,Park,Pier,Harbor / Marina,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space
61,Downtown Toronto,0.0,Park,Pier,Harbor / Marina,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space
62,Downtown Toronto,0.0,Park,Pier,Harbor / Marina,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space
68,North York,0.0,Construction & Landscaping,Park,Trail,Dog Run,Field,Fast Food Restaurant,Farmers Market,Fireworks Store,Farm,Donut Shop


Cluster 2

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
154,Scarborough,1.0,Pharmacy,Women's Store,Donut Shop,Fish & Chips Shop,Fireworks Store,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
155,Scarborough,1.0,Pharmacy,Women's Store,Donut Shop,Fish & Chips Shop,Fireworks Store,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
156,Scarborough,1.0,Pharmacy,Women's Store,Donut Shop,Fish & Chips Shop,Fireworks Store,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
157,Scarborough,1.0,Pharmacy,Women's Store,Donut Shop,Fish & Chips Shop,Fireworks Store,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


Cluster 3

In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Etobicoke,2.0,Golf Course,Women's Store,Donut Shop,Fish & Chips Shop,Fireworks Store,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
16,Etobicoke,2.0,Golf Course,Women's Store,Donut Shop,Fish & Chips Shop,Fireworks Store,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
17,Etobicoke,2.0,Golf Course,Women's Store,Donut Shop,Fish & Chips Shop,Fireworks Store,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
18,Etobicoke,2.0,Golf Course,Women's Store,Donut Shop,Fish & Chips Shop,Fireworks Store,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
19,Etobicoke,2.0,Golf Course,Women's Store,Donut Shop,Fish & Chips Shop,Fireworks Store,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


Cluster 4

In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,3.0,Park,Grocery Store,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Women's Store,Dog Run
9,North York,3.0,Burger Joint,Park,Coffee Shop,Women's Store,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Farm
105,North York,3.0,Coffee Shop,Nightclub,Park,Women's Store,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant
106,North York,3.0,Coffee Shop,Nightclub,Park,Women's Store,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant
121,Central Toronto,3.0,Park,Women's Store,Donut Shop,Fish & Chips Shop,Fireworks Store,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
122,Central Toronto,3.0,Park,Women's Store,Donut Shop,Fish & Chips Shop,Fireworks Store,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


Cluster 5

In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,4.0,Coffee Shop,Bakery,Theater,Boat or Ferry,Gym / Fitness Center,Italian Restaurant,Hotel,Gym Pool,Ice Cream Shop,Mexican Restaurant
3,North York,4.0,Clothing Store,Furniture / Home Store,Cosmetics Shop,Sporting Goods Shop,American Restaurant,Men's Store,Toy / Game Store,Food Court,Lingerie Store,Café
4,North York,4.0,Clothing Store,Furniture / Home Store,Cosmetics Shop,Sporting Goods Shop,American Restaurant,Men's Store,Toy / Game Store,Food Court,Lingerie Store,Café
5,Queen's Park,4.0,Coffee Shop,Café,Sandwich Place,Pharmacy,Italian Restaurant,Park,Bookstore,Sushi Restaurant,Middle Eastern Restaurant,Portuguese Restaurant
6,Queen's Park,4.0,Coffee Shop,Café,Sandwich Place,Pharmacy,Italian Restaurant,Park,Bookstore,Sushi Restaurant,Middle Eastern Restaurant,Portuguese Restaurant
10,East York,4.0,Fast Food Restaurant,Pizza Place,Pet Store,Rock Climbing Spot,Pharmacy,Gym / Fitness Center,Gastropub,Café,Athletics & Sports,Breakfast Spot
11,East York,4.0,Fast Food Restaurant,Pizza Place,Pet Store,Rock Climbing Spot,Pharmacy,Gym / Fitness Center,Gastropub,Café,Athletics & Sports,Breakfast Spot
12,Downtown Toronto,4.0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Bakery,Italian Restaurant,Ramen Restaurant,Burger Joint,Japanese Restaurant
13,Downtown Toronto,4.0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Bakery,Italian Restaurant,Ramen Restaurant,Burger Joint,Japanese Restaurant
14,North York,4.0,Pizza Place,Fast Food Restaurant,Asian Restaurant,Sushi Restaurant,Latin American Restaurant,Japanese Restaurant,Grocery Store,Mediterranean Restaurant,Falafel Restaurant,Farm


<h4> This will help a person relocating a business or house to find a similar neighborhood in the Toronto<h4>